In [111]:
# Imports and plotting setups
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

import sys, os
sys.path.insert(0, '../../')
sys.path.insert(0, '../')

import math_funcs

from cycler import cycler

default_cycler = (cycler(color=['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', '#BBBBBB']) +
                cycler(linestyle=['-', '--', ':', '-.', '-', '--', ':']))

plt.rc('lines', linewidth=1)
plt.rc('axes', prop_cycle=default_cycler)

plt.rcParams["font.family"] = "Times"
plt.rcParams["font.size"] = 8

plt.rcParams['figure.figsize'] = (3.5, 2.5)
plt.rcParams['figure.dpi'] = 600

plt.rcParams['text.usetex'] = True

In [112]:
# Constants
c_0	= 299792458 # m/s
e = 2.718281828
epsilon_0 = 8.854E-12
mu_0 = 1.257E-06
k = 1.38E-23

In [113]:
# Howell 2021 Model parameters
T_surf = 104 # K
T_cond_base = 230.4 # K
T_conv_iso = 251.6 # K
brittle_cryosphere_depth = 10000 * 1000 # m
surface_background_radiation_temperature = 10**4 # K

convective_cryosphere_depth = 5.8 * 1000 # m
T_melt = 273.13 # K

total_cryosphere_depth = brittle_cryosphere_depth + convective_cryosphere_depth

In [114]:
# Mission Parameters
max_BER = 10**-5
bit_rate = 1e3 # bps
link_BW = 10e3 #3.43e3 # Hz

In [115]:
# Engineering/Assumed Parameters
transmitted_power = 0.4 # W

def antenna_directivity(T):
    directivity_in_100K_ice = 4.64 # dB
    directivity_in_273K_ice = 4.46 # dB
    m, b = math_funcs.linear_fit(
        100, math_funcs.db_2_power(directivity_in_100K_ice), 
        273, math_funcs.db_2_power(directivity_in_273K_ice))
    return m * T + b

def radiation_efficiency(T):
    rad_eff_in_273K_ice = 0.228 # dB
    rad_eff_in_100K_ice = 0.223 # dB
    m, b = math_funcs.linear_fit(
        100, rad_eff_in_100K_ice, 
        273, rad_eff_in_273K_ice)
    return m * T + b

matching_efficiency = 0.952
carrier_frequency = 1e7#413*10**6 # Hz

In [124]:
# Propagation path modeling equations
def temperature_at_depth(depth):
    if depth > total_cryosphere_depth:
        return T_melt
    elif depth > brittle_cryosphere_depth:
        return T_conv_iso
    else:
        m, b = math_funcs.linear_fit(
            0, T_surf, 
            brittle_cryosphere_depth, T_cond_base)
        return m * depth + b

def ice_epsilon_relative(T):
    T = T - 273.13 # convert from Kelvin to Celsius
    real = 3.1884 + 0.00091*T
    imag = 10**(-3.0129 + 0.0123*T)
    return real - 1j*imag

def ice_wave_number(T):
    return 2 * np.pi * carrier_frequency * np.sqrt(epsilon_0 * mu_0) * np.sqrt(ice_epsilon_relative(T))

def wavelength_column_path_loss_calc_linear(top_column, bottom_column):
    iter_depth = 10 # about 20 wavelengths
    depths = np.arange(top_column, bottom_column + iter_depth, iter_depth)
    linear_path_loss = 1 # real units of power
    for col_depth in depths:
        k_s = ice_wave_number(temperature_at_depth(col_depth))
        linear_path_loss *= np.e**(2 * iter_depth * np.imag(k_s))

    return linear_path_loss

def wavelength_column_path_loss_calc(top_column, bottom_column, debug=False):

    linear_path_loss = wavelength_column_path_loss_calc_linear(top_column, bottom_column)

    path_length = bottom_column - top_column
    space_path_loss = (1 / (2 * \
        np.real(ice_wave_number(temperature_at_depth(bottom_column))) \
            * (path_length)))**2
    total_path_loss = linear_path_loss * space_path_loss

    if debug:
        return 10*np.log10(total_path_loss), 10*np.log10(space_path_loss), 10*np.log10(linear_path_loss)
    else:
        return total_path_loss

def received_power(T_puck, top_column, bottom_column):
    path_loss = wavelength_column_path_loss_calc(top_column, bottom_column)
    return transmitted_power * matching_efficiency**2 * radiation_efficiency(T_puck)**2\
        * path_loss * antenna_directivity(T_puck)**2

def noise_power(T_puck, top_column, bottom_column, receiver_noise_temperature=0):
    path_loss = wavelength_column_path_loss_calc_linear(top_column, bottom_column)
    return link_BW * k * (T_puck + receiver_noise_temperature) \
        + radiation_efficiency(T_puck) * link_BW * k * path_loss * surface_background_radiation_temperature

def CNR_per_bit(received_power, noise_power):
    return (bit_rate/link_BW) * (received_power/noise_power)

def bit_error_rate(CNR_per_bit):
    if 10*np.log10(CNR_per_bit) > 15:
        return 10**-40
    elif CNR_per_bit > 4*np.log(2):
        return np.e**(-1 * 1 * (CNR_per_bit - 2 * np.log(2)))
    else:
        return np.e**(-1 * 2 * ((np.sqrt(CNR_per_bit) - np.sqrt(np.log(2)))**2))

def bit_error_rate_of_next_receiver(last_puck_depth, next_puck_path_length, debug=False):
    puck_depth = last_puck_depth + next_puck_path_length
    T_puck = temperature_at_depth(puck_depth)

    P_r = received_power(T_puck, last_puck_depth, puck_depth)
    P_n = noise_power(T_puck, 0, puck_depth)

    BER = bit_error_rate(CNR_per_bit(P_r, P_n))
    if debug:
        return BER, puck_depth, T_puck, P_r, P_n
    else:
        return BER

def SNR_of_next_receiver(last_puck_depth, next_puck_path_length, receiver_noise_temperature=0):
    puck_depth = last_puck_depth + next_puck_path_length
    T_puck = temperature_at_depth(puck_depth)

    P_r = received_power(T_puck, last_puck_depth, puck_depth)
    P_n = noise_power(T_puck, 0, puck_depth, receiver_noise_temperature)

    return P_r/P_n

def optimize_puck_depth(last_puck_depth):
    # Goal function
    goal_func = lambda next_puck_path_length: bit_error_rate_of_next_receiver(last_puck_depth, next_puck_path_length) - max_BER

    a = 1
    b = total_cryosphere_depth - last_puck_depth

    # Bound the answer to b if the
    if np.sign(goal_func(a)) and np.sign(goal_func(b)):
        return b

    # Using a binary search optimization (since I know the bit error rate monotonically increases with depth)
    return scipy.optimize.bisect(goal_func, a, b)

def optimize_puck_depth_simple_SNR_bound(last_puck_depth):
    # Goal function
    goal_func = lambda next_puck_path_length: SNR_of_next_receiver(last_puck_depth, next_puck_path_length, 600) - 10**(3/10)

    a = 1
    b = total_cryosphere_depth - last_puck_depth

    # Bound the answer to b if the
    if np.sign(goal_func(a)) and np.sign(goal_func(b)):
        return b

    # Using a binary search optimization (since I know the bit error rate monotonically increases with depth)
    return scipy.optimize.bisect(goal_func, a, b)

x = [100, 1000, 10.4e3, 20e3]
test_temp = 104
ice_wave_number(test_temp), ice_epsilon_relative(test_temp), \
    [wavelength_column_path_loss_calc(0, i, True) for i in x], \
    [10*np.log10(received_power(test_temp, 0, i)) for i in x], \
    [10*np.log10(noise_power(temperature_at_depth(i), 0, i, 600)) for i in x], \
    [10*np.log10(received_power(temperature_at_depth(i), 0, i)/ noise_power(temperature_at_depth(i), 0, i, 600)) for i in x], \
    [10*np.log10(CNR_per_bit(received_power(test_temp, 0, i), noise_power(600, 0, i))) for i in x], \
    [bit_error_rate_of_next_receiver(0, i) for i in x]


((0.3651406993230382-4.85450880781075e-07j),
 (3.0344917000000002-8.06865228237526e-06j),
 [(-37.27027025059173, -37.26980641932817, -0.000463831263560619),
  (-57.2740807320867, -57.269821235241466, -0.004259496845232996),
  (-77.65461866384335, -77.61064276217965, -0.043975901663696845),
  (-83.37540870099902, -83.29073391711667, -0.08467478388236446)],
 [-45.43448975784618,
  -65.43830023934115,
  -85.81883817109781,
  -91.53962820825348],
 [-153.92524731455563,
  -153.92811050015706,
  -153.95803264511517,
  -153.98862440335574],
 [108.4907563999444, 88.48979869312052, 68.13907416510415, 62.448764822220866],
 [98.43293725210305,
  78.43215646492487,
  58.083287996186364,
  52.394889796499974],
 [1e-40, 1e-40, 1e-40, 1e-40])

In [117]:
T_puck = 150 # K
path_loss = 1
10*np.log10(transmitted_power * matching_efficiency**2 * radiation_efficiency(T_puck)**2\
        * path_loss * antenna_directivity(T_puck)**2)

-8.206981288287347

In [118]:
# Loop to calculate the number of pucks and all of their stats
total_depth = 0
inital_guess = 2000
puck_depths = [0]
i = 0
next_puck_depth = 0

while next_puck_depth < total_cryosphere_depth:
    last_puck_depth = puck_depths[i]
    next_puck_depth = optimize_puck_depth(last_puck_depth)
    puck_depths.append(next_puck_depth)

puck_depths, len(puck_depths) - 1

([0, 10005800.0], 1)

In [119]:
# Loop to calculate the number of pucks and all of their stats
total_depth = 0
puck_depths = [0]
i = 0
next_puck_depth = 0

while next_puck_depth < total_cryosphere_depth:
    last_puck_depth = puck_depths[i]
    next_puck_depth = optimize_puck_depth_simple_SNR_bound(last_puck_depth)
    puck_depths.append(next_puck_depth)

puck_depths, len(puck_depths) - 1

([0, 10005800.0], 1)

In [120]:
SNR_of_next_receiver(0, 25e3, 600)

1123341.5098334078